In [ ]:
from IPython.core.debugger import prompt
from autorag.data.legacy.corpus import llama_text_node_to_parquet
from autorag.data.legacy.qacreation import make_single_content_qa, generate_qa_llama_index
from autorag.utils import cast_corpus_dataset
from gitdb.fun import chunk_size
from llama_index.legacy.llms import OpenAI


In [1]:
from autorag.utils.preprocess import cast_corpus_dataset 

[12/05/24 09:49:30] INFO     [config.py:58] >> PyTorch version 2.5.1+cu124 available.                  ]8;id=30093;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\datasets\config.py\config.py]8;;\:]8;id=634411;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\datasets\config.py#58\58]8;;\

E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.wa

In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter

documents = SimpleDirectoryReader("../data/genesis/", recursive=True).load_data()

In [3]:
from autorag.data.legacy.corpus.llama_index import llama_text_node_to_parquet
nodes = TokenTextSplitter().get_nodes_from_documents(documents=documents, chunk_size=256, chunk_overlap=64)
corpus_df = llama_text_node_to_parquet(nodes)
corpus_df = cast_corpus_dataset(corpus_df)
corpus_df.to_parquet('../data/test_dataset/corpus_new.parquet')

# 질문생성

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
prompt = """다음을 참고로 제네시스 차량 매뉴얼에 관한 내용을 만들었습니다.
매뉴얼을 보고 할 만한 질문을 만드세요.
반드시 제네시스 차량과 관련한 질문이어야 합니다.
만약 주어진 매뉴얼 내용이 제네시스 차량과 관련되지 않았다면,
'제네시스 차량과 관련 없었습니다.'라고 질문을 만드세요.

매뉴얼:
{{text}}

생성할 질문 개수: {{num_questions}}

예시:
[Q]: 제네시스 차량의 타이어 공기압은 얼마인가요?
[A]: 제네시스 차량의 타이어 공기압은 35 PSI입니다.

제네시스 차량과 관련이 없는 매뉴얼의 경우 예시:
[Q]: 제네시스 차량과 관련 없었습니다.
[A]: 제네시스 차량과 관련 없었습니다.
"""

In [8]:
import pandas as pd
from llama_index.llms.openai import OpenAI
from autorag.data.legacy.qacreation import make_single_content_qa, generate_qa_llama_index

corpus_df = pd.read_parquet('../data/test_dataset/corpus_new.parquet', engine='pyarrow')
llm = OpenAI(model='gpt-4o-mini', temperature=0.5)

qa_df = make_single_content_qa(corpus_df=corpus_df, content_size=10, qa_creation_func=generate_qa_llama_index,
                               llm=llm, prompt=prompt, question_num_per_content=2)

0it [00:00, ?it/s]

[12/05/24 09:50:52] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=667574;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=996218;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[12/05/24 09:50:53] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=199549;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=70450;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=313510;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=200821;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[12/05/24 09:50:54] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=820180;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=957035;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[12/05/24 09:50:57] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=456684;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=562354;file://E:\TailorLink\SKN03-FINAL-6Team\TailorLink_LLM\doc_retrieval_qa\.venv\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

1it [00:07,  7.93s/it]


In [10]:
qa_df.to_parquet('../data/test_dataset/corpus_new_test.parquet')